### **Speed Camera using OpenCV**
**01416500 - Computer Visions**

Author:
- Piyawud Koonmanee 63110119
- Lapin Buranaassavakul 63110110

In [1]:
# regenerate folder for new output
!rmdir /S /Q "Results/Fail/Total_Vehicle/" "Results/Fail/Exceeded/"
!mkdir "Results/Fail/Total_Vehicle/" "Results/Fail/Exceeded/"

In [2]:
# Import necessary libraries
import cv2
import numpy as np
from math import hypot
from time import time

##### **User Input Numbers**

In [ ]:
# speed limit
limit = 60 # km/h

# detection distance
distance = 8.7 # meter

# region of interest
region_of_interest = [[150,476],[727,538],[877,219],[694,197]]

##### **Helper Function**

In [3]:
class CarTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_point = {}
        self.id_count = 0
        self.s1 = np.zeros((1,1000))
        self.s2 = np.zeros((1,1000))
        self.s = np.zeros((1,1000))
        self.f = np.zeros(1000)
        self.capf = np.zeros(1000)
        self.count = 0
        self.exceeded = 0
        
        # write head of the file
        file = open("Results/Fail/Speed_Record.txt","w")
        file.write("\tID\t|\tSPEED\n—————————\n")
        file.close()

    def update(self, objects_rect):
        object_box_id = []

        # center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2
            
            # check for the same object
            same_object = False

            for id, pt in self.center_point.items():
                dist = hypot(cx-pt[0], cy-pt[1])

                # threshold for the same bounding box
                if dist < 82:
                    self.center_point[id] = (cx, cy)
                    object_box_id.append([x, y, w, h, id])
                    same_object = True

                    # start timer
                    if (self.center_point[id][1] >= 325 and self.center_point[id][1] <= 335):
                        self.s1[0,id] = time()

                    # stop timer and calculate the time taken
                    if (self.center_point[id][1] >= 148 and self.center_point[id][1] <= 158):
                        self.s2[0,id] = time()
                        self.s[0,id] = self.s2[0,id] - self.s1[0,id]

                    # capture the car
                    if (self.center_point[id][1] >= 127 and self.center_point[id][1] <= 137):
                        self.f[id]=1

            # detect new object
            if same_object is False:
                self.center_point[self.id_count] = (cx, cy)
                object_box_id.append([x, y, w, h, self.id_count])
                self.id_count += 1
                self.s[0,self.id_count] = 0
                self.s1[0,self.id_count] = 0
                self.s2[0,self.id_count] = 0

        # assign new ID to an object
        new_center_point = {}
        for obj_bb_id in object_box_id:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_point[object_id]
            new_center_point[object_id] = center

        self.center_point = new_center_point.copy()
        return object_box_id

    # speed calculation
    def get_speed(self,id):
        if (self.s[0,id] != 0):
            s = distance / self.s[0, id] # in m/s
        else:
            s = 0

        return round((s*3.6),2) # convert to km/h

    # save vehicle data
    def capture(self,img,x,y,h,w,sp,id):
        if(self.capf[id] == 0):
            self.capf[id] = 1
            self.f[id]=0
            crop_img = img[y-5:y+h+5, x-5:x+w+5]
            n = str(id)+"_speed_"+str(sp)
            file = 'Results/Fail/Total_Vehicle/' + n + '.jpg'
            cv2.imwrite(file, crop_img)
            self.count += 1
            datas = open("Results/Fail/Speed_Record.txt", "a")

            if(sp > limit):
                file2 = 'Results/Fail/Exceeded/' + n + '.jpg'
                cv2.imwrite(file2, crop_img)
                datas.write("\t" + str(id) + "\t|\t" + str(sp) + "\t<--- exceeded\n")
                self.exceeded+=1
            else:
                datas.write("\t" + str(id) + "\t|\t" + str(sp) + "\n")
            datas.close()

    # write summary
    def end(self):
        file = open("Results/Fail/Speed_Record.txt", "a")
        file.write("\n\n———————————\n")
        file.write("\t\tSummary\n")
        file.write("———————————\n")
        file.write("Total Vehicles :\t" + str(self.count)+"\n")
        file.write("Speeding Vehicles :\t" + str(self.exceeded))
        file.close()

##### **Main Function**

In [4]:
# read video
cap = cv2.VideoCapture("datasets/Fail.mp4")
tracker = CarTracker()

# initialize background subtractor
sub_bg = cv2.createBackgroundSubtractorMOG2(history = 300, varThreshold = 700)

# kernels
opening_kernals = np.ones((10,10),np.uint8)
closing_kernals = np.ones((16,16),np.uint8)
dilate_kernals = np.ones((10,10),np.uint8)

# write Video
framerate = 30.0
codec = cv2.VideoWriter_fourcc('M','J','P','G')
out = cv2.VideoWriter('Results/Fail/Tracking.avi', codec, framerate, (960,540))

while cap.isOpened():
    ret,frame = cap.read()

    if not ret:
        # close window when done
        break

    # resize video by half (1920,1080) -> (960,540)
    frame = cv2.resize(frame, None, fx = 0.5, fy = 0.5)

    # region of interest
    point = np.float32(region_of_interest)
    rect = cv2.boundingRect(point)
    x,y,w,h = rect
    croped = frame[y:y+h, x:x+w].copy()

    point = point - point.min(axis = 0)
    mask = np.zeros(croped.shape[:2], np.uint8)
    cv2.drawContours(mask, [point.astype(int)], -1, (255, 255, 255), -1, cv2.LINE_AA)
    roi = cv2.bitwise_and(croped, croped, mask = mask)

    # rotate
    height, width, _ = roi.shape
    m = cv2.getRotationMatrix2D((width-1/2,height+1/2),17.2,1)
    roi = cv2.warpAffine(roi, m, (width, height+60))
 
    # masking
    mask = sub_bg.apply(roi)
    _, thres = cv2.threshold(mask, 100, 255, cv2.THRESH_BINARY)
    mask1 = cv2.morphologyEx(thres, cv2.MORPH_OPEN, opening_kernals) # remove noise
    mask2 = cv2.morphologyEx(mask1, cv2.MORPH_CLOSE, closing_kernals) # fill hole
    dilate = cv2.dilate(mask2, dilate_kernals, iterations = 3)

    # contour detection
    contour,_ = cv2.findContours(dilate,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    detection = []

    for i, cnt in enumerate(contour):
        area = cv2.contourArea(cnt)
        if area > 400: 
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(roi,(x,y),(x+w,y+h),(0,255,0),3)
            detection.append([x,y,w,h])

    # object tracking
    boxes_ids = tracker.update(detection)
    for box_id in boxes_ids:
        x,y,w,h,id = box_id

        # draw centroid
        # cx = (x+x+w) // 2
        # cy = (y+y+h) // 2
        # cv2.circle(roi, (cx,cy), 5, (255,0,0), -1)
        # cv2.putText(roi, str((cx,cy)), (cx-15,cy-15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0),2)
            
        if(tracker.get_speed(id) < limit):
            # draw green bounding box
            label = "[" + str(id) + "] " + str(tracker.get_speed(id))
            (wb, hb), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1, 2)
            cv2.rectangle(roi, (x, y-25), (x+wb, y), (0, 255, 0), -1)
            cv2.putText(roi,label, (x,y-10), cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
            cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 255, 0), 3)
        else:
            # draw orange bounding box
            label = "[" + str(id) + "] " + str(tracker.get_speed(id))
            (wb, hb), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1, 2)
            cv2.rectangle(roi, (x, y-25), (x+wb, y), (0, 165, 255), -1)
            cv2.putText(roi,label,(x, y-10),cv2.FONT_HERSHEY_PLAIN, 1,(255, 255, 255),2)
            cv2.rectangle(roi, (x, y), (x+w, y+h), (0, 165, 255), 3)

        s = tracker.get_speed(id)
        if (tracker.f[id] == 1 and s != 0):
            tracker.capture(roi, x, y, h, w, s, id)

    # draw detection lines
    cv2.line(roi, (0,330), (960,330), (0, 0, 255), 1) # start line
    cv2.line(roi, (0,153), (960,153), (0, 0, 255), 1) # stop line
    cv2.line(roi, (0,132), (960,132), (0, 255, 255), 1) # capture line

    # display video
    cv2.imshow("Video", roi)
    cv2.imshow("Threshold", dilate)
    out.write(cv2.resize(roi,(960,540))) # resize and write video
    
    # stop video on ESC (8)
    if cv2.waitKey(8) == 27:
        break

tracker.end()
cap.release()
out.release()
cv2.destroyAllWindows()